In [1]:
import importlib
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from flearn.utils.model_utils import read_data
import os
import importlib
import random
import numpy as np

from flearn.models import client
from flearn.models import group
from flearn.trainers import grouprox
from flearn.trainers import fedprox
from flearn.trainers import fedavg

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
params = {}
#DATASETS = ['sent140', 'nist', 'shakespeare', 'mnist', 'synthetic_iid', 'synthetic_0_0', 'synthetic_0.5_0.5', 'synthetic_1_1']

"""
Parameters that need to be manually set
"""

#params['dataset'] = 'sent140'
params['dataset'] = 'nist'
params['model'] = 'mlp'
#params['model'] = 'stacked_lstm'
params['optimizer'] = 'grouprox'
params['drop_percent'] = 0

if params['dataset'].startswith('mnist'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.03
  params['mu'] = 1
if params['dataset'] == 'nist':
  params['model_params'] = (26,)
  params['learning_rate'] = 0.003
  params['mu'] = 1
if params['dataset'] == 'sent140':
  params['model_params'] = (25, 2, 100) # seq_len, num_classes, num_hidden
  params['learning_rate'] = 0.3
  params['mu'] = 0.01
if params['dataset'].startswith('synthetic'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.01
  params['mu'] = 1

if params['optimizer']  == 'fedavg':
  pass
if params['optimizer']  == 'fedprox':
  pass
if params['optimizer']  == 'grouprox':
  params['proximal'] = False
  params['randomly'] = False
  params['allow_empty'] = True
  params['evenly'] = False
  params['num_group'] = 5
  params['min_clients'] = 2


"""
Common parameters
"""
params['num_rounds'] = 200
params['num_epochs'] = 20
params['batch_size'] = 10
params['eval_every'] = 1
params['clients_per_round'] = 20 ######################################################
params['seed'] = 233
params['agg_lr'] = 0.01


""" Set the output CSV file name """
info = '{}-{}-{}'.format(params['optimizer'], params['dataset'], params['model'])
info2 = ''
suffix = '.csv'
if params['optimizer'] == 'fedavg':
  pass
if params['optimizer'] == 'fedprox':
  pass
if params['optimizer'] == 'grouprox':
  info2 = '-NumG{:1d}-Agglr{:.4}-Proximal{:.1}'.format(
    params['num_group'], params['agg_lr'], str(params['proximal']))
  if params['randomly'] == True:
    info2 = info2 + '-Randomly{:.1}-Evenly{:.1}'.format(str(params['randomly']), str(params['evenly']))
  elif params['allow_empty'] == True:
    info2 = info2 + '-Empty{:.1}'.format(str(params['allow_empty']))
  elif params['allow_empty'] == False:
    info2 = info2 + '-Evenly{:.1}'.format(str(params['evenly']))

params['export_filename'] = info + info2 + suffix

print(params['export_filename'])

grouprox-nist-mlp-NumG5-Agglr0.01-ProximalT-EmptyT.csv


In [3]:
"""
Some preparations
"""

# read data
train_path = os.path.join('data', params['dataset'], 'data', 'train')
test_path = os.path.join('data', params['dataset'], 'data', 'test')
dataset = read_data(train_path, test_path)

# Load model
if params['dataset'].startswith('synthetic'):  # all synthetic datasets use the same model
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'synthetic', params['model'])
elif params['dataset'].startswith('mnist'):
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'mnist', params['model'])
else:
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', params['dataset'], params['model'])
mod = importlib.import_module(model_path)
learner = getattr(mod, 'Model')

# Set seeds
random.seed(1 + params['seed'])
np.random.seed(12 + params['seed'])
tf.set_random_seed(123 + params['seed'])

# Set server
if params['optimizer'] == 'fedavg':
  server = fedavg.Server(params, learner, dataset)  
if params['optimizer'] == 'fedprox':
  server = fedprox.Server(params, learner, dataset)
if params['optimizer'] == 'grouprox':
  server = grouprox.Server(params, learner, dataset)

Using Group prox to Train
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
200 Clients in Total
415258 Parameters in Total
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


Pre-training take 3.175800085067749s seconds
SVD take 0.8088743686676025s seconds
(100, 415258) (415258, 3)
Calculate Cossim matrix take 0.1857466697692871s seconds
Clustering take 0.05171680450439453s seconds
Clustering Results: Counter({0: 25, 1: 22, 2: 19, 3: 17, 4: 17})
Clustering Inertia: 1.3325695991516113


In [4]:
server.train()

Training with 20 workers ---


Group 0, clients ['f_00154', 'f_00033', 'f_00074']
Group 1, clients ['f_00018', 'f_00107', 'f_00098', 'f_00177', 'f_00007']
Group 2, clients ['f_00182', 'f_00012', 'f_00152', 'f_00061']
Group 3, clients ['f_00005', 'f_00146', 'f_00125', 'f_00037']
Group 4, clients ['f_00170', 'f_00180', 'f_00080', 'f_00130']


Epoch:  30%|███████████████████████                                                      | 6/20 [00:00<00:00, 53.76it/s]

Group 0
At round 0 accuracy: 0.4458128078817734
At round 0 training accuracy: 0.391758560650029
At round 0 training loss: 1.7083789346111562
Group 1
At round 0 accuracy: 0.37349397590361444
At round 0 training accuracy: 0.3649322879543835
At round 0 training loss: 1.8041862008066918
Group 2
At round 0 accuracy: 0.3838383838383838
At round 0 training accuracy: 0.3688639551192146
At round 0 training loss: 1.9114284743043295
Group 3
At round 0 accuracy: 0.3474320241691843
At round 0 training accuracy: 0.33166368515205724
At round 0 training loss: 1.8117142595417384
Group 4
At round 0 accuracy: 0.5222222222222223
At round 0 training accuracy: 0.5148578811369509
At round 0 training loss: 1.7020223318221341
At round 0 mean test accuracy: 0.41476091476091476 mean train accuracy: 0.3941903659947628
The groups difference are: [0.93582042 0.90971989 0.92222375 0.86537209 0.9360785 ]
Begin group  0 training


Epoch:   0%|                                                                                     | 0/20 [00:00<?, ?it/s]

...

Begin group  2 training


Epoch:   0%|                                                                                     | 0/20 [00:00<?, ?it/s]

Begin group  3 training


Epoch:  25%|███████████████████▎                                                         | 5/20 [00:00<00:00, 43.59it/s]

Begin group  4 training


In [5]:
from collections import Counter

group_list = server.group_list
clients = server.clients
g = group_list[-1]
c_list = []
for c in clients:
  if c.group == g:
    c_list.append(c)

cnt = Counter()
for c in c_list:
  cnt += Counter(c.eval_data['y'])
print(cnt)
print(sum(cnt.values()))

Counter({0.0: 126, 1.0: 111, 2.0: 80, 9.0: 43})
360


In [6]:
nd1 = np.arange(10).reshape(5,2)
nd2 = np.arange(10).reshape(5,2)
nd3 = np.array([3.]*10).reshape(5,2)
print((nd2+1.)/2.)
#print(nd2)

print(np.linalg.norm(nd2, axis=1, keepdims=True).shape)

[[0.5 1. ]
 [1.5 2. ]
 [2.5 3. ]
 [3.5 4. ]
 [4.5 5. ]]
(5, 1)
